SAN
====

**Side Adapter Network for Open-Vocabulary Semantic Segmentation**

 * Paper: https://arxiv.org/pdf/2302.12242

![SAN Overview](../assets/san_overview.jpg)

![SAN Architecture](../assets/san_arch.jpg)

![SAN using CLIP-attn to predict mask](../assets/san_attn-mask.jpg)